## What is the protein domain profile of genes from the same module ? 


In [1]:
from intermine.webservice import Service
import numpy as np
import scipy.io
import seaborn as sns
from scipy import stats, optimize, interpolate
import pandas as pd
from collections import defaultdict 
import math
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm
from scipy import stats
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import os, fnmatch

## Selecting the modules to evaluate the protein domains 

In [ ]:
pathways=[ 'galactose degradation','phospholipid biosynthesis', "ethanol degradation"]
biological_processes=["cell budding"]

In [ ]:
service = Service('https://yeastmine.yeastgenome.org/yeastmine/service', token = '11d7t1Y007Fa97c82f27')
query = service.new_query("Pathway")
query.add_view("name", "genes.symbol")
query.add_constraint("name", "=", "galactose degradation", code = "A")

data_toy=defaultdict(dict)

for row,counter in zip(query.rows(),np.arange(0,len(query.rows()))):

     data_toy['gene-name'][counter]=row['genes.symbol']

data_toy_pd=pd.DataFrame(data_toy)
data_gal_pathway=data_toy_pd.drop_duplicates()

In [2]:
service = Service('https://yeastmine.yeastgenome.org/yeastmine/service', token = '11d7t1Y007Fa97c82f27')
query = service.new_query("Gene")
query.add_constraint("goAnnotation.ontologyTerm.parents", "GOSlimTerm")
query.add_view(
    "symbol", "goAnnotation.evidence.code.annotType",
    "goAnnotation.ontologyTerm.parents.name"
)
query.add_constraint("goAnnotation.qualifier", "!=", "NOT", code = "C")
query.add_constraint("goAnnotation.qualifier", "IS NULL", code = "D")
query.add_constraint("goAnnotation.evidence.code.annotType", "=", "manually curated", code = "F")
query.add_constraint("goAnnotation.ontologyTerm.parents.name", "=", "cell budding", code = "G")
query.set_logic("(C or D) and F and G")

data_toy=defaultdict(dict)

for row,counter in zip(query.rows(),np.arange(0,len(query.rows()))):

     data_toy['gene-name'][counter]=row["symbol"]
     data_toy['evidence'][counter]=row["goAnnotation.evidence.code.annotType"]
     data_toy['annotation'][counter]=row["goAnnotation.ontologyTerm.parents.name"]


data_toy_pd=pd.DataFrame(data_toy)
data_cell_budding=data_toy_pd.drop_duplicates()



In [3]:
data_cell_budding

,gene-name,evidence,annotation
0,BUD6,manually curated,cell budding
2,MOB2,manually curated,cell budding
5,SLA2,manually curated,cell budding
6,BUD4,manually curated,cell budding
7,TGL3,manually curated,cell budding
9,CMD1,manually curated,cell budding
10,PPH22,manually curated,cell budding
11,PPH21,manually curated,cell budding
12,STE20,manually curated,cell budding
15,PEA2,manually curated,cell budding


In [ ]:
## Build a protein list of pairs from that module 